In [1]:
from dotenv import dotenv_values
from SPARQLWrapper import SPARQLWrapper, POST, JSON
import requests

### Selección de BOE

In [2]:
boe_id = "BOE-A-2023-8320"

In [3]:
env_config = dotenv_values("../.env")

In [4]:
sparql = SPARQLWrapper(
    f"http://{env_config['GRAPHDB_HOST']}:{env_config['GRAPHDB_PORT']}"
    f"/repositories/{env_config['GRAPHDB_REPOSITORY']}"
)
sparql.setMethod(POST)

In [5]:
PREFIXES = """
PREFIX  :     <http://www.semanticweb.org/hackathon/ontology/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  xml:  <http://www.w3.org/XML/1998/namespace>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
"""

In [6]:
def get_titulos(boe_ids):
    sparql.setQuery(PREFIXES + f"""
    SELECT ?entradaBOE ?titulo
    WHERE {{
    ?entradaBOE rdf:type :EntradaBOE .
    ?entradaBOE :identificador "{boe_id}" .
    ?entradaBOE :titulo ?titulo .
    }}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"][0]

### Obtención anteriores y posteriores

In [7]:
def get_referenced_boes(boe_id):
    query = f"""
                PREFIX  :     <http://www.semanticweb.org/hackathon/ontology/>
                PREFIX  rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

                SELECT ?entradaBOE ?boe_anterior_correccion ?boe_posterior_correccion ?boe_anterior ?boe_posterior
                WHERE {{
                ?entradaBOE rdf:type :EntradaBOE .
                ?entradaBOE :identificador "{boe_id}" .
                
                OPTIONAL {{
                    ?entradaBOE :publica ?boe_anterior .
                    FILTER EXISTS {{ ?entradaBOE :publica ?boe_anterior }}
                }}
                
                OPTIONAL {{
                    ?entradaBOE :sePublica ?boe_posterior .
                    FILTER EXISTS {{ ?entradaBOE :sePublica ?boe_posterior }}
                }}
                
                OPTIONAL {{
                    ?entradaBOE :corrigeErrores ?boe_anterior_correccion .
                    FILTER EXISTS {{ ?entradaBOE :corrigeErrores ?boe_anterior_correccion }}
                }}
                
                OPTIONAL {{
                    ?entradaBOE :seCorrigenErrores ?boe_posterior_correccion .
                    FILTER EXISTS {{ ?entradaBOE :seCorrigenErrores ?boe_posterior_correccion }}
                }}
                }}
            """

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql.addParameter('Content-Type', 'application/sparql-query')
    results = sparql.query().convert()
    anteriores = []
    posteriores = []
    for result in results["results"]["bindings"]:
        entradaBOE = result["entradaBOE"]["value"].split("/")[-1]
        try:
            anteriores.append(result["boe_anterior"]["value"].split("/")[-1])
        except:
            pass
        try:
            anteriores.append(result["boe_anterior_correccion"]["value"].split("/")[-1])
        except:
            pass
        try:
            posteriores.append(result["boe_posterior"]["value"].split("/")[-1])
        except:
            pass
        try:
            posteriores.append(result["boe_posterior_correccion"]["value"].split("/")[-1])
        except:
            pass
    
    return {'anteriores': anteriores, 'posteriores': posteriores}

In [8]:
refs = get_referenced_boes(boe_id)
print(f"Entrada seleccionada del BOE: {boe_id}")

print("\nBOEs anteriores:")
for ref in refs['anteriores']:
    print(f"\t*{ref}: {get_titulos(ref)['titulo']['value']}")

print("\nBOEs posteriores:")
for ref in refs['posteriores']:
    print(f"\t*{ref}: {get_titulos(ref)['titulo']['value']}")

Entrada seleccionada del BOE: BOE-A-2023-8320

BOEs anteriores:
	*BOE-A-2023-7897: Resolución de 31 de marzo de 2023, de la Subsecretaría, por la que se corrigen errores de la de 23 de marzo de 2023, por la que se resuelve el concurso específico convocado por la Resolución de 20 de enero de 2023.

BOEs posteriores:
	*BOE-A-2023-8500: Resolución de 31 de marzo de 2023, de la Subsecretaría, por la que se corrigen errores de la de 23 de marzo de 2023, por la que se resuelve el concurso específico convocado por la Resolución de 20 de enero de 2023.
